<a href="https://colab.research.google.com/github/Karula-k/triplet_loss_speaker_recognition/blob/main/tripletloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=629cc1a270be979816c98ca5f9b7486b219d0151f099414440467609b1b40667
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=12011375564075612934d1feb74ac1e3aaa434d76c2303bcae6e6cfb4a5829b3
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [2]:
import os
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import random
import librosa
import umap.umap_ as umap
import itertools
import gc
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from tensorflow import summary

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/self-data"

In [ ]:
TRIPLET_ALPHA = 0.1

### Prepare the Data

In [ ]:
class data_preparation:
  def __init__(self,dir,extractType,split_val):
    self.dir = dir
    self.extractType = extractType
    self.split_val = split_val
  def extract_features(self,audio_file):
    """Extract MFCC features from an audio file, shape=(TIME, MFCC)."""
    waveform, sample_rate = sf.read(audio_file)
     # Convert to mono-channel.
    if len(waveform.shape) == 2:
        waveform = librosa.to_mono(waveform.transpose())

    # Convert to 16kHz.
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr= 16000)
    if self.extractType == "concanate":
      features_mfcc = librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=40).transpose()
      features_LPC = librosa.core.lpc(waveform,order=39)
      return np.vstack((features_LPC,features_mfcc))
    else:
      features = librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=40)
      return features.transpose()
  def get_dataset(self):
    """Get the dict from speaker to list of utterances for dataset."""
    data = []
    label = []
    wav_files = glob.glob(os.path.join(self.dir,"*/*.wav"))
    spk_to_utts = dict()
    for wav_file in wav_files:
        basename = os.path.basename(wav_file)
        spk = basename
        split_name = basename.split("-")
        spk = split_name[0]
        if spk not in spk_to_utts:
            spk_to_utts[spk] = [self.extract_features(wav_file)]
        else:
            spk_to_utts[spk].append(self.extract_features(wav_file))
    return spk_to_utts

  def split_data(self):
    dictionary = self.get_dataset()
    train = {}
    test = {}
    for spk, data_spk in dictionary.items():
      total_data = len(data_spk)
      train[spk] = data_spk[:-int(self.split_val*total_data)]
      test[spk] = data_spk[-int(self.split_val*total_data):]
    del dictionary
    return train, test

In [ ]:
def extract_features(audio_file):
    """Extract MFCC features from an audio file, shape=(TIME, MFCC)."""
    waveform, sample_rate = sf.read(audio_file)

    # Convert to mono-channel.
    if len(waveform.shape) == 2:
        waveform = librosa.to_mono(waveform.transpose())

    # Convert to 16kHz.
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr= 16000)
    features = librosa.feature.mfcc(
        y=waveform, sr=sample_rate, n_mfcc=40)
    return features.transpose()

In [ ]:
def extract_feature_concanate(audio_file):
    """Extract MFCC features from an audio file, shape=(TIME, MFCC)."""
    waveform, sample_rate = sf.read(audio_file)

    # Convert to mono-channel.
    if len(waveform.shape) == 2:
        waveform = librosa.to_mono(waveform.transpose())

    # Convert to 16kHz.
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr= 16000)
    features_mfcc = librosa.feature.mfcc(
        y=waveform, sr=sample_rate, n_mfcc=40).transpose()
    features_LPC = librosa.core.lpc(waveform,order=39)
    return np.vstack((features_LPC,features_mfcc))

### Embeding model choice

In [ ]:
class embeding_model_selection:
  def __init__(self,embeding_model,shape):
     self.shape = shape
     self.embeding_model = self.make_model() if(embeding_model == 'dense') else self.make_model_LSTM()
  def make_model(self):
    return tf.keras.models.Sequential([
    tf.keras.layers.Dense(64,input_shape = self.shape),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(64)
  ])
  def make_model_LSTM(self):
    return tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64,input_shape = self.shape,return_sequences=True),
    tf.keras.layers.LSTM(64,return_sequences=True),
    tf.keras.layers.LSTM(64)])
  def model_combination(self):
    input_anchor = tf.keras.layers.Input(shape=self.shape)
    input_positive = tf.keras.layers.Input(shape=self.shape)
    input_negative = tf.keras.layers.Input(shape=self.shape)
    embedding_anchor = self.embeding_model(input_anchor)
    embedding_positive = self.embeding_model(input_positive)
    embedding_negative = self.embeding_model(input_negative)

    output = tf.keras.layers.concatenate([embedding_anchor, embedding_positive, embedding_negative], axis=1)
    net = tf.keras.models.Model([input_anchor, input_positive, input_negative], output)
    return net


### Triplet extraction

In [ ]:
# Easy Triplets are those where D(A,P) + margin < D(A,N), thus L = 0.
# Hard Triplets are those where D(A,N) < D(A,P).
# Semi-Hard Triplets are those where D(A,P) < D(A,N) < D(A,P) + margin.

In [ ]:
def get_triplet_random(data,shape,embedding_model):
    """Get a triplet of anchor/pos/neg samples."""
    pos_spk, neg_spk = random.choices(list(data.keys()), k=2)
    while len(data[pos_spk]) < 2 or len(data[neg_spk])< 2:
        pos_spk, neg_spk = random.choices(list(data.keys()),k= 2)
    anchor_utt, pos_utt = random.choices(data[pos_spk], k=2)
    neg_utt = random.choices(data[neg_spk], k=1)[0]
    return (anchor_utt.reshape(shape), pos_utt.reshape(shape), neg_utt.reshape(shape))

In [ ]:
def get_triplet_semi_hard(data,shape,embedding_model):
    """Get a triplet of anchor/pos/neg samples."""

    # Select two different speakers for positive and negative samples
    pos_spk, neg_spk = random.sample(list(data.keys()), k=2)

    # Randomly select anchor and positive utterances from the chosen speaker
    anchor_utt, pos_utt = random.sample(data[pos_spk], k=2)

    # Randomly select a negative utterance from the chosen negative speaker
    neg_utt = random.choice(data[neg_spk])

    # Convert the anchor, positive, and negative utterances to tensors
    anchor_utt_tensor = embedding_model.predict(anchor_utt.reshape(shape))
    pos_utt_tensor = embedding_model.predict( pos_utt.reshape(shape))
    neg_utt_tensor =  embedding_model.predict( neg_utt.reshape(shape))
    # print(anchor_utt,neg_utt,pos_utt)

    # Calculate cosine similarities
    neg_cosine_sim = cos(anchor_utt_tensor, neg_utt_tensor)
    pos_cosine_sim = cos(anchor_utt_tensor, pos_utt_tensor)

    print(neg_cosine_sim, pos_cosine_sim)

    # Ensure the negative utterance is farther from the anchor than the positive utterance
    # while neg_cosine_sim.item() >= pos_cosine_sim.item():
    while pos_cosine_sim.item() >= neg_cosine_sim.item() and neg_cosine_sim.item() >= (pos_cosine_sim.item()+TRIPLET_ALPHA) :
        print(neg_cosine_sim.item(), pos_cosine_sim.item())
        # Randomly select new anchor and positive utterances
        anchor_utt, pos_utt = random.sample(data[pos_spk], k=2)
        neg_utt = random.choice(data[neg_spk])
        # Convert the new positive utterance to a tensor
        anchor_utt_tensor = embedding_model.predict(anchor_utt.reshape(shape))
        pos_utt_tensor = embedding_model.predict( pos_utt.reshape(shape))
        neg_utt_tensor =  embedding_model.predict( neg_utt.reshape(shape))
        # Calculate the new cosine similarity for negative utterance
        neg_cosine_sim = cos(anchor_utt_tensor, neg_utt_tensor)
        pos_cosine_sim = cos(anchor_utt_tensor, pos_utt_tensor)
    print(neg_cosine_sim.item(), pos_cosine_sim.item())
    return (anchor_utt.reshape(shape), pos_utt.reshape(shape), neg_utt.reshape(shape))

In [ ]:
def get_triplet_hard(data,shape,embedding_model):
    """Get a triplet of anchor/pos/neg samples."""

    # Select two different speakers for positive and negative samples
    pos_spk, neg_spk = random.sample(list(data.keys()), k=2)

    # Randomly select anchor and positive utterances from the chosen speaker
    anchor_utt, pos_utt = random.sample(data[pos_spk], k=2)

    # Randomly select a negative utterance from the chosen negative speaker
    neg_utt = random.choice(data[neg_spk])

    # Convert the anchor, positive, and negative utterances to tensors
    anchor_utt_tensor = embedding_model.predict(anchor_utt.reshape(shape))
    pos_utt_tensor = embedding_model.predict( pos_utt.reshape(shape))
    neg_utt_tensor =  embedding_model.predict( neg_utt.reshape(shape))
    # print(anchor_utt,neg_utt,pos_utt)

    # Calculate cosine similarities
    neg_cosine_sim = cos(anchor_utt_tensor, neg_utt_tensor)
    pos_cosine_sim = cos(anchor_utt_tensor, pos_utt_tensor)

    print(neg_cosine_sim, pos_cosine_sim)

    # Ensure the negative utterance is farther from the anchor than the positive utterance
    while neg_cosine_sim.item() >= pos_cosine_sim.item():
    # while pos_cosine_sim.item() >= neg_cosine_sim.item() and neg_cosine_sim.item() >= (pos_cosine_sim.item()+TRIPLET_ALPHA) :
        print(neg_cosine_sim.item(), pos_cosine_sim.item())
        # Randomly select new anchor and positive utterances
        anchor_utt, pos_utt = random.sample(data[pos_spk], k=2)
        neg_utt = random.choice(data[neg_spk])
        # Convert the new positive utterance to a tensor
        anchor_utt_tensor = embedding_model.predict(anchor_utt.reshape(shape))
        pos_utt_tensor = embedding_model.predict( pos_utt.reshape(shape))
        neg_utt_tensor =  embedding_model.predict( neg_utt.reshape(shape))
        # Calculate the new cosine similarity for negative utterance
        neg_cosine_sim = cos(anchor_utt_tensor, neg_utt_tensor)
        pos_cosine_sim = cos(anchor_utt_tensor, pos_utt_tensor)
    print(neg_cosine_sim.item(), pos_cosine_sim.item())
    return (anchor_utt.reshape(shape), pos_utt.reshape(shape), neg_utt.reshape(shape))

In [ ]:
# data = data_preparation(path,'mfcc',0.8)
# x_train,x_test =  data.split_data()
# embeding_model = embeding_model_selection('LSTM',(None,40))
# embd = embeding_model.embeding_model
# net = embeding_model.model_combination()
# get_triplet_hard(x_train,(1,313,40),embd)
# a= x_test['prabowo'][1]
# a.shape
# net.layers[3].predict(a.reshape(1,313,40))

### Loss choice

In [ ]:
def triplet_loss(y_true, y_pred):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    cosine_similarity_positive = tf.keras.losses.cosine_similarity(anchor, positive)
    cosine_similarity_negative = tf.keras.losses.cosine_similarity(anchor, negative)

    positive_dist = tf.square(cosine_similarity_positive)
    negative_dist = tf.square(cosine_similarity_negative)
    # tf.print('positive distance',positive_dist,'\nnegative distance',negative_dist, '\nsubtract',positive_dist-negative_dist)
    loss = tf.maximum(tf.subtract(positive_dist,tf.add(negative_dist, TRIPLET_ALPHA)), 0.0)
    return loss

In [ ]:
def triplet_loss_penalty(y_true,y_pred):
    alpha = 1
    m =0.1
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    cosine_similarity_positive = tf.keras.losses.cosine_similarity(anchor, positive)
    cosine_similarity_negative = tf.keras.losses.cosine_similarity(anchor, negative)

    positive_dist = tf.square(cosine_similarity_positive)
    negative_dist = tf.square(cosine_similarity_negative)
    if(positive_dist>negative_dist+TRIPLET_ALPHA):
      loss = tf.add(tf.maximum(tf.subtract(positive_dist,tf.add(negative_dist, TRIPLET_ALPHA)), 0.0),tf.multiply(tf.cast(alpha, tf.float32),tf.divide(tf.subtract(negative_dist,positive_dist),2)))
      # loss = tf.maximum(positive_dist-negative_dist+ TRIPLET_ALPHA, 0.0)+alpha*(negative_dist-positive_dist)/2
      return loss
    elif(positive_dist- negative_dist>TRIPLET_ALPHA*(m-1) and positive_dist- negative_dist<TRIPLET_ALPHA*(2*m-1)):
      loss = tf.subtract(tf.maximum(tf.subtract(positive_dist,tf.add(negative_dist, TRIPLET_ALPHA)), 0.0),tf.multiply(tf.cast(alpha, tf.float32),tf.divide(tf.subtract(negative_dist,positive_dist),2)))
      # loss = tf.maximum(positive_dist-negative_dist+ TRIPLET_ALPHA, 0.0)-alpha*(negative_dist-positive_dist)/2
      return loss
    else:
      # loss = tf.maximum(positive_dist - negative_dist + TRIPLET_ALPHA, 0.0)
      loss = tf.maximum(tf.subtract(positive_dist,tf.add(negative_dist, TRIPLET_ALPHA)), 0.0)
      return loss

In [ ]:
#https://github.com/shivsondhi/Triplet-Loss/blob/master/triplet_loss_functions.py
def triplet_loss_func(y_true, y_pred, alpha=0.3):
	'''
	Used directly as loss function
		Inputs:
					y_true: True values of classification. (y_train)
					y_pred: predicted values of classification.
					alpha: Distance between positive and negative sample, arbitrarily
						   set to 0.3

		Returns:
					Computed loss

		Function:
					--Implements triplet loss using tensorflow commands
					--The following function follows an implementation of Triplet-Loss
					  where the loss is applied to the network in the compile statement
					  as usual.
	'''
	anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

	positive_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), -1)
	negative_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)), -1)

	loss_1 = tf.add(tf.subtract(positive_dist, negative_dist), alpha)
	loss = tf.reduce_sum(tf.maximum(loss_1, 0.0))

	return loss

### Data save and prepare

In [ ]:
def generate_and_save_triplets(data, num_triplets, save_path,get_triplet,shape,embedding_model):
    triplets = []

    for _ in range(num_triplets):
        # anchor_utt, pos_utt, neg_utt = get_triplet(data)
        anchor_utt, pos_utt, neg_utt = get_triplet(data,shape,embedding_model)
        triplets.append((anchor_utt, pos_utt, neg_utt))

    np.savez(save_path, triplets=triplets)

In [ ]:
def data_generator_from_triplets(triplets,shape,batch_size=32):
    if len(shape)>2:
        shape = (batch_size,shape[1],shape[2])
    else:
        shape = (batch_size,shape[1])
    while True:
        indices = np.random.choice(len(triplets), size=batch_size, replace=False)
        x_anchors = np.zeros(shape)
        x_positives = np.zeros(shape)
        x_negatives = np.zeros(shape)

        batch_triplets = [triplets[i] for i in indices]
        for i in range(0, batch_size):

          x_anchors[i] = batch_triplets[i][0]
          x_positives[i] = batch_triplets[i][1]
          x_negatives[i] = batch_triplets[i][2]
          # anchor_batch = np.array([triplet[0] for triplet in batch_triplets])
          # pos_batch = np.array([triplet[1] for triplet in batch_triplets])
          # neg_batch = np.array([triplet[2] for triplet in batch_triplets])
        y = np.zeros((batch_size, 3*64))
        yield [x_anchors, x_positives, x_negatives], y

### Train

### Evaluation

In [ ]:
def speaker_modeling(train,model,shape):
    """ Mean validation"""
    list_model_spk = {}
    for speaker in train.keys():
        model_speaker = []
        for data_spk in train[speaker]:
            # model_spk = random.choices(train[speaker], k=1)[0]
            features = data_spk.reshape(shape)
            data = model.layers[3].predict(features)
            if data is None:
                continue
            model_speaker.append(data)
        try:
          if model_speaker == []:
            print(speaker)
            continue
          model_speaker = np.mean(model_speaker, axis=0)
          list_model_spk[speaker] = model_speaker
        except:
          continue
    return list_model_spk

In [ ]:
def test_model(test, list_model_spk,model_embd,shape):
    y_truth = []
    y_hat_list = []
    embeding_list = []
    for label, files in test.items():
        for wav in files:
            model_speaker = model_embd.layers[3].predict(wav.reshape(shape))
            if model_speaker is None:
              print('model_speaker', label)
              continue
            max_label = ""
            max_values = 0
            for  model in list_model_spk:
                # print('model',model)
                values = cos(model_speaker, list_model_spk[model])
                # print('model_speaker',list_model_spk[model_speaker])
                # print('model',model)
                if values > max_values:
                    max_values = values
                    max_label = model
            y_truth.append(label)
            y_hat_list.append(max_label)
            embeding_list.append(model_speaker)

    return y_truth, y_hat_list,embeding_list

In [ ]:
def calculate_accuracy(y_true, y_pred):
    correct = 0
    total = 0
    for true, pred in zip(y_true, y_pred):
        if y_pred =="":
          continue
        elif true == pred:
            correct += 1
            total+=1
        else:
          total+=1
    accuracy = correct / total

    return accuracy

In [ ]:
def prepare_targets(y):
 le = LabelEncoder()
 le.fit(y)
 y_train_enc = le.transform(y)
 return y_train_enc

In [ ]:
def save_chart(y_predict,y_test,embeding,name):
  embd = np.array(embeding)
  embd = embd.reshape(356,64)
  #UMAP
  reduced_embeddings_umap = umap.UMAP(n_neighbors=15, min_dist=0.3, metric='correlation').fit_transform(embd)
  #PCA
  pca = PCA(n_components=2)
  reduced_embeddings_pca= pca.fit_transform(embd)
  y_test = prepare_targets(y_test)
  y_predict = prepare_targets(y_predict)
  plt.scatter(reduced_embeddings_umap[:, 0], reduced_embeddings_umap[:, 1], c=y_test)
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/graphic_image/{name} y_test umap.png')
  plt.scatter(reduced_embeddings_pca[:, 0], reduced_embeddings_pca[:, 1], c=y_test)
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/graphic_image/{name} y_test pca.png')
  plt.scatter(reduced_embeddings_umap[:, 0], reduced_embeddings_umap[:, 1], c=y_predict)
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/graphic_image/{name} y_predict umap.png')
  plt.scatter(reduced_embeddings_pca[:, 0], reduced_embeddings_pca[:, 1], c=y_predict)
  plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/graphic_image/{name} y_predict pca.png')

In [ ]:
def main (model_type,extract_type,triplet,triplet_loss):
  batch_size = 32
  epochs = 10
  num_triplets_to_generate = 1000
  steps_per_epoch=num_triplets_to_generate // batch_size
  name = f'{model_type} {extract_type} {triplet} {triplet_loss.__name__}'
  triplets_save_path = f'/content/drive/MyDrive/Colab Notebooks/npz_data/{name} test.npz'
  tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=f'./Graph/{name}', histogram_freq=0,
          write_graph=True, write_images=True)

  data = data_preparation(path,extract_type,0.8)
  x_train,x_test =  data.split_data()

  if model_type == "LSTM" and extract_type == "MFCC":
    shape = (1,313,40)
    shape_model = (None,40)
  elif model_type == "LSTM" and extract_type =="concanate":
    shape = (1,314,40)
    shape_model = (None,40)
  elif model_type =='dense' and extract_type =="MFCC" :
    shape  = (1,12520)
    shape_model  = (None,12520)
  elif model_type =='dense' and extract_type =="concanate" :
    shape  = (1,12560)
    shape_model  = (None,12560)

  embeding_model = embeding_model_selection(model_type,shape_model)
  if (triplet == "semi-hard"):
    triplet_type = get_triplet_semi_hard
  elif (triplet == "hard"):
    triplet_type = get_triplet_hard
  else:
    triplet_type = get_triplet_random
  embdm = embeding_model.embeding_model
  net = embeding_model.model_combination()
  generate_and_save_triplets(x_train, num_triplets_to_generate, triplets_save_path,triplet_type,shape,embdm)
  triplets_data = np.load(triplets_save_path, allow_pickle=True)
  triplets = triplets_data['triplets']
  triplets_generator = data_generator_from_triplets(triplets, shape,batch_size)
  net.compile(loss=triplet_loss, optimizer='adam')
  history = net.fit(triplets_generator,epochs=epochs,steps_per_epoch=steps_per_epoch, callbacks=[tbCallBack])
  model_h = speaker_modeling(x_train,net,shape)
  tested_train = test_model(x_train,model_h,net,shape)
  tested = test_model(x_test,model_h,net,shape)
  accuracy_train = calculate_accuracy(tested_train[0], tested_train[1])
  accuracy = calculate_accuracy(tested[0], tested[1])
  # save_chart(tested[0],tested[1],tested[2],name)
  summary_writer = summary.create_file_writer(f'./Graph/{name}/train')
  with summary_writer.as_default():
    summary.scalar('Accuracy', accuracy_train,step=1)
    summary.scalar('Accuracy', accuracy,step=1)
  del x_train
  del x_test

  return history,accuracy,accuracy_train

In [ ]:
a,b,c = main('dense','MFCC','semi-hard',triplet_loss_func)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 39ms/step
[[0.776337]] [[0.698477]]
0.7763370275497437 0.698477029800415
1/1 [==============================] - 0s 34ms/step
[[0.65548337]] [[0.59755504]]
0.6554833650588989 0.5975550413131714
1/1 [==============================] - 0s 33ms/step
[[0.61224514]] [[0.72628486]]
0.6122451424598694 0.7262848615646362
1/1 [==============================] - 0s 35ms/step
[[0.8070799]] [[0.85661083]]
0.8070799112319946 0.8566108345985413
1/1 [==============================] - 0s 35ms/step
[[0.6518226]] [[0.70781225]]
0.6518226265907288 0.7078122496604919
1/1 [==============================] - 0s 32ms/step
[[0.7533126]] [[0.7548851]]
0.7533125877380371 0.7548850774765015
1/1 [==============================] - 0s 36ms/step
[[0.7135179]] [[0.91345084]]
0.7135179042816162 0.9134508371353149
1/1 [==============================] - 0s 21ms/step
[[0.6186237]] [[0.62020206]]
0.618623673915863 0.6202020645141602
1

In [ ]:
# model_type,extract_type,triplet = 'dense','MFCC','hard'
# batch_size = 32
# epochs = 100
# num_triplets_to_generate = 1000
# steps_per_epoch=num_triplets_to_generate // batch_size
# triplets_save_path = '/content/drive/MyDrive/Colab Notebooks/npz_data/triplets semi hard split2.npz'

# data = data_preparation(path,extract_type,0.8)
# x_train,x_test =  data.split_data()

# if model_type == "LSTM" and extract_type == "MFCC":
#   shape = (1,313,40)
#   shape_model = (None,40)
# elif model_type == "LSTM" and extract_type =="concanate":
#   shape = (1,314,40)
#   shape_model = (None,40)
# elif model_type =='dense' and extract_type =="MFCC" :
#   shape  = (1,12520)
#   shape_model  = (None,12520)
# elif model_type =='dense' and extract_type =="concanate" :
#   shape  = (1,12560)
#   shape_model  = (None,12560)

# embeding_model = embeding_model_selection(model_type,shape_model)
# if (triplet == "semi-hard"):
#   triplet_type = get_triplet_semi_hard
# elif (triplet == "hard"):
#   triplet_type = get_triplet_hard
# else:
#   triplet_type = get_triplet_random
# embdm = embeding_model.embeding_model
# net = embeding_model.model_combination()
# generate_and_save_triplets(x_train, num_triplets_to_generate, triplets_save_path,triplet_type,shape,embdm)
# triplets_data = np.load(triplets_save_path, allow_pickle=True)
# triplets = triplets_data['triplets']
# triplets_generator = data_generator_from_triplets(triplets, batch_size,shape)
# net.compile(loss=triplet_loss, optimizer='adam')
# triplets_generator = data_generator_from_triplets(triplets, shape,batch_size)
# history = net.fit(triplets_generator,epochs=epochs,steps_per_epoch=steps_per_epoch, callbacks=[tbCallBack])
# model_h = speaker_modeling(x_train,net,shape)
# tested = test_model(x_test,model_h,net,shape)
# accuracy = calculate_accuracy(tested[0], tested[1])
# history.history['loss'][-1]

In [ ]:
def hyperparameter_search(model_types, extract_types, triplet_types, loss_types, resume=False):
    csv_file_path = '/content/drive/MyDrive/Colab Notebooks/results_csv/hyperparameter_results.csv'
    best_accuracy = 0
    best_params = {}
    results = []

    # Load previously saved results if resuming
    if resume:
        with open(csv_file_path, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                accuracy = float(row['Accuracy'])
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        "Model Type": row['Model Type'],
                        "Extract Type": row['Extract Type'],
                        "Triplet Type": row['Triplet Type'],
                        "Loss Type": row['Loss Type']
                    }
                results.append(row)

    # Create or open a CSV file for saving the results
    with open(csv_file_path, 'a+', newline='') as csvfile:
        fieldnames = ['Model Type', 'Extract Type', 'Triplet Type', 'Loss Type', 'Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not resume:
            writer.writeheader()

        for model_type, extract_type, triplet_type,loss_type in itertools.product(model_types, extract_types, triplet_types,loss_types):
            print(model_type, extract_type, triplet_type)
            if resume:

                # Skip combinations that have already been processed
                if any(result['Model Type'] == model_type and
                       result['Extract Type'] == extract_type and
                       result['Triplet Type'] == triplet_type and 'Loss Type' == loss_type.__name__ for result in results):
                    continue

            history, accuracy = main(model_type, extract_type, triplet_type, loss_type)

            # Save the results to the CSV file
            writer.writerow({
                'Model Type': model_type,
                'Extract Type': extract_type,
                'Triplet Type': triplet_type,
                'Loss Type': loss_type.__name__,
                'Accuracy': accuracy
            })

            # Update best results
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    "Model Type": model_type,
                    "Extract Type": extract_type,
                    "Triplet Type": triplet_type,
                    "Loss Type": loss_type.__name__
                }
            gc.collect()

    return best_params, best_accuracy

In [ ]:
def hyperparameter_search_dummy(model_types, extract_types, triplet_types, loss_types, resume=False):
    csv_file_path = '/content/drive/MyDrive/Colab Notebooks/results_csv/hyperparameter_results.csv'
    best_accuracy = 0
    best_params = {}
    results = []

    # Load previously saved results if resuming
    if resume:
        with open(csv_file_path, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                accuracy = float(row['Accuracy'])
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        "Model Type": row['Model Type'],
                        "Extract Type": row['Extract Type'],
                        "Triplet Type": row['Triplet Type'],
                        "Loss Type": row['Loss Type']
                    }
                results.append(row)

    # Create or open a CSV file for saving the results
    with open(csv_file_path, 'a+', newline='') as csvfile:
        fieldnames = ['Model Type', 'Extract Type', 'Triplet Type', 'Loss Type', 'Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not resume:
            writer.writeheader()
        n = 0
        for model_type, extract_type, triplet_type,loss_type in itertools.product(model_types, extract_types, triplet_types,loss_types):
            print(model_type, extract_type, triplet_type,loss_type.__name__)
            if resume:
                # Skip combinations that have already been processed
                if any(result['Model Type'] == model_type and
                       result['Extract Type'] == extract_type and
                       result['Triplet Type'] == triplet_type and result['Loss Type'] == loss_type.__name__ for result in results):
                  print('skipped')
                  continue
                # if any(result['Model Type'] == model_type and
                #        result['Extract Type'] == extract_type and
                #        result['Triplet Type'] == triplet_type and 'Loss Type' == loss_type.__name__ for result in results):
                #     continue
            n+=1
            accuracy = random.randint(0,100)

            # Save the results to the CSV file
            writer.writerow({
                'Model Type': model_type,
                'Extract Type': extract_type,
                'Triplet Type': triplet_type,
                'Loss Type': loss_type.__name__,
                'Accuracy': accuracy
            })
            if(n==5):
              break
            # Update best results
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    "Model Type": model_type,
                    "Extract Type": extract_type,
                    "Triplet Type": triplet_type,
                    "Loss Type": loss_type.__name__
                }
            gc.collect()

    return best_params, best_accuracy

In [ ]:
model_types = ["LSTM", "dense"]
extract_types = ["MFCC", "concatenate"]
triplet_types = ["semi-hard", "hard", "random"]
loss_types = [triplet_loss,triplet_loss_penalty,triplet_loss_func]

best_params, best_accuracy = hyperparameter_search_dummy(model_types,extract_types,triplet_types,loss_types,True)
print("Best Hyperparameters:")
print("Model Type:", best_params["Model Type"])
print("Extract Type:", best_params["Extract Type"])
print("Triplet Type:", best_params["Triplet Type"])
print("Loss Type:", best_params["Loss Type"])
print("Best Accuracy:", best_accuracy)

LSTM MFCC semi-hard triplet_loss
skipped
LSTM MFCC semi-hard triplet_loss_penalty
skipped
LSTM MFCC semi-hard triplet_loss_func
skipped
LSTM MFCC hard triplet_loss
skipped
LSTM MFCC hard triplet_loss_penalty
skipped
LSTM MFCC hard triplet_loss_func
skipped
LSTM MFCC random triplet_loss
skipped
LSTM MFCC random triplet_loss_penalty
skipped
LSTM MFCC random triplet_loss_func
skipped
LSTM concatenate semi-hard triplet_loss
skipped
LSTM concatenate semi-hard triplet_loss_penalty
skipped
LSTM concatenate semi-hard triplet_loss_func
skipped
LSTM concatenate hard triplet_loss
skipped
LSTM concatenate hard triplet_loss_penalty
skipped
LSTM concatenate hard triplet_loss_func
skipped
LSTM concatenate random triplet_loss
skipped
LSTM concatenate random triplet_loss_penalty
skipped
LSTM concatenate random triplet_loss_func
skipped
dense MFCC semi-hard triplet_loss
skipped
dense MFCC semi-hard triplet_loss_penalty
skipped
dense MFCC semi-hard triplet_loss_func
skipped
dense MFCC hard triplet_loss
s

In [ ]:
model_types = ["LSTM", "dense"]
extract_types = ["MFCC", "concatenate"]
triplet_types = ["semi-hard", "hard", "random"]
loss_types = [triplet_loss,triplet_loss_penalty,triplet_loss_func]

best_params, best_accuracy = hyperparameter_search(model_types,extract_types,triplet_types,loss_types,False)
print("Best Hyperparameters:")
print("Model Type:", best_params["Model Type"])
print("Extract Type:", best_params["Extract Type"])
print("Triplet Type:", best_params["Triplet Type"])
print("Loss Type:", best_params["Loss Type"])
print("Best Accuracy:", best_accuracy)

NameError: ignored

In [ ]:
domain = [(0,1),(0,1),(0,2),(0,2),(0,2),(0,1)]

In [ ]:
def fitness_function(hyperparameter):
    hyperparameter = encode(hyperparameter)
    accuracy = main(*hyperparameter)
    return accuracy

In [ ]:
def mutation(domain, schedule, prob):
  gene = random.randint(0, len(domain) - 1)
  #print(gene)
  mutant = schedule
  if random.random() < prob: # 0.02, 0.05
    if schedule[gene] != domain[gene][0]:
      mutant = schedule[0:gene] + [schedule[gene] - 1] + schedule[gene + 1:]
    else:
      if schedule[gene] != domain[gene][1]:
        mutant = schedule[0:gene] + [schedule[gene] + 1] + schedule[gene + 1:]
  return mutant

In [ ]:
def crossover(domain, individual1, individual2):
  gene = random.randint(1, len(domain) - 2)
  # print(gene)
  return individual1[0:gene] + individual2[gene:]

ada 20 gen --> save gene nya id nya generation,quarter,current task,  5 nya dikerjain -->  save berdasarkan id gen nya

component tambahan


>worker = yang bisa ngerjain fitness fuction berdasarkan id
worker harus bisa kerja sama sama load

>csv yang lain nyimpan history last id

>save indiviual dulu baru ke fitness pakai id + pake tru false trained

In [ ]:
def worker(generation,population,csv_filename,quarter,resume):
  costs = []
  for individual in population:
    cost = fitness_function(individual)
    costs.append(cost)
    save_population_to_csv(csv_filename,cost, individual,quarter, generation,resume)
  return costs

In [ ]:
import random

def genetic_algorithm(domain, fitness_function, csv_filename,resume, population_size=20, elitism=0.2,
                      number_of_generations=5, mutation_probability=0.05):
    current_generation,current_quarter,population = load_population_from_csv(csv_filename)

    if population is None:
        population = []
        for i in range(population_size):
            individual = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
            population.append(individual)
            save_population_to_csv(csv_filename,0, individual,current_quarter, current_generation,resume)
    ##else klo ada population

    number_elitism = int(elitism * population_size)
    while current_generation < number_of_generations:
        costs = []
        while current_quarter < 4:
          starts = 5 *current_quarter*current_generation
          costs.extend(worker(current_generation,population[starts:starts+5],csv_filename,current_quarter,resume))
          current_quarter +=1

        costs.sort(reverse=True)
        ordered_individuals = [individual for (cost, individual) in costs]
        population = ordered_individuals[0:number_elitism]

        while len(population) < population_size:
            i1 = random.randint(0, number_elitism)
            i2 = random.randint(0, number_elitism)
            new_individual = crossover(domain, ordered_individuals[i1], ordered_individuals[i2])
            new_individual_mutation = mutation(domain, new_individual, mutation_probability)
            population.append(new_individual_mutation)
        current_generation +=1

    return costs[0][1]

In [ ]:
def dummy_fitness(args):
  return random.randint(0,9)

In [ ]:
dummy_fitness(9)

8

In [ ]:
domain = [(0, 9), (0, 9),(0, 9),(0, 9)]  # Example domain
result = genetic_algorithm(domain, dummy_fitness, '/content/drive/MyDrive/Colab Notebooks/results_csv/population_data.csv')
print("Best solution:", result)

Best solution: [8, 2, 7, 3]


In [16]:
import csv
import os

In [17]:
def save_population_to_csv(csv_filename,cost, population,quarter,train, current_generation,resume):
    with open(csv_filename, 'a+', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        if not resume:
        # Write the name row with "Generation" and "Fitness" columns
          csv_writer.writerow(["Generation", "Fitness","Quarter","Train"] + [f"Gene_{i}" for i in range(len(population))])

        # Write the data for each individual in the population
        row = [current_generation, cost,quarter,train] + population
        csv_writer.writerow(row)

In [26]:
current_gen = 10  # Replace with the current generation number
population = [1, 2, 3]  # Replace with your population data
cost = 10
quarter  =1
csv_filename = "/content/drive/MyDrive/Colab Notebooks/results_csv/test.csv"  # Replace with your desired CSV filename

save_population_to_csv(csv_filename,cost, population,quarter, current_gen,True)

In [24]:
def load_population_from_csv(csv_filename):
    if os.path.exists(csv_filename):
      population = []
      generation = 0
      quarter = 0
      with open(csv_filename, 'r') as csv_file:
          csv_reader = csv.reader(csv_file)
          next(csv_reader)
          for row in csv_reader:
              individual = [int(value) for value in row[4:]]
              generation = int(row[0])
              quarter = int(row[2])
              train = int(row[3])
              population.append([train,individual])
      return generation,quarter,population
    else:
        return None,0

In [28]:
load_population_from_csv(csv_filename)[0]

10

In [ ]:
def encode(gene):
  model_types = {0:"LSTM",1:"dense",2:"CNN"}
  extract_types = {0:"MFCC",1:"concatenate"}
  triplet_types = {0:"semi-hard",1:"hard",2:"random"}
  loss_types = {0:"triplet_loss",1:"triplet_loss_penalty",2:"triplet_loss_ecludian"}
  splits = {0:0.7,1:0.8,2:0.9}
  index_dict = [model_types,extract_types,triplet_types,loss_types,splits]
  return [dict_place[index] for index,dict_place in zip(gene,index_dict)]

In [ ]:
def decode(gene):
  model_types = {"LSTM":0,"dense":1,"CNN":2}
  extract_types = {"MFCC":0,"concatenate":1}
  triplet_types = {"semi-hard":0,"hard":1,"random":2}
  loss_types = {"triplet_loss":0,"triplet_loss_penalty":1,"triplet_loss_ecludian":2}
  splits = {0.7:0,0.8:1,0.9:2}
  index_dict = [model_types,extract_types,triplet_types,loss_types,splits]
  return [dict_place[index] for index,dict_place in zip(gene,index_dict)]

1.  buat encoder decoder
2.  resume after 5 coder (quarter)
3.  make it zero if its not yet the trained

### Graphic Shown

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=Graph

## to-do


1.   ~~pecah train dan test~~
2.   ~~buat triplet offline karena memori gk kuat aka kentang~~
      
    refrensi [link](https://arxiv.org/pdf/2007.02200.pdf)
3.   ~~predict per label & di mean axis 1 biar shape nya tetap 313*40~~
4.   ~~test prediksi dengan sk-learn~~
5.  ~~perbaiki nilai loss~~   ! can be done more better
6.  ganti embeding LSTM
7.  metode untuk feature extrasis di ganti
9.  perbaiki penulisan
8.  buat seknario

<hr>


Treiplet Loss Scenario
  - MFCC
  - LPC & MFCC
  - HARD
  - Semi - Hard
  - Easy random
  - Split 70
  - Split 80
  - Split 90
  - LSTM
  - Adgard
  - Hyperparameter tuning (grid search, keras tuner)

<hr>


Basic Scenario
  - MFCC
  - MFCC LPC
  - Split 70
  - Split 80
  - Split 90
  - LSTM
  - Adgard

- tensorboard
- gambar persebaran data
- accuracy
- continue in csv